<a href="https://colab.research.google.com/github/pierredumontel/plane-classification/blob/main/notebooks/import_def_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Données tabulaires (reg logisitique -> pas approprié pour vecteur de coordonnées d'images) 
Pytorch (Facebook) vs tensorflow (Google) (pour les NN complexes)

Pytorch + facile a utilisé en termes de structure de réseaux

Keras : sur couche des 2 autres 

pep8 

In [1]:
!curl -O https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/archives/fgvc-aircraft-2013b.tar.gz
!tar zxf fgvc-aircraft-2013b.tar.gz
!mv fgvc-aircraft-2013b dataset 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2625M  100 2625M    0     0  9132k      0  0:04:54  0:04:54 --:--:-- 14.0M


In [2]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd 

In [3]:
pd.__version__

'1.3.5'

In [36]:
DATA_DIR = pathlib.Path('dataset/data')

### Make assumptions about number of spaces (here=2)

In [5]:
manufacturer_df = pd.read_csv(DATA_DIR / 'images_manufacturer_train.txt',sep=' ',
            names=['image_id','m1','m2'],
            usecols=['image_id','m1','m2'],#usecols for v1.4 compatibility
            dtype={'image_id':str}) #ids are not int but string 
manufacturer_df['manufacturer']=manufacturer_df['m1']+' '+manufacturer_df['m2']
manufacturer_df['manufacturer'].unique()

array([nan, 'British Aerospace', 'Lockheed Corporation',
       'Douglas Aircraft', 'McDonnell Douglas', 'de Havilland',
       'Lockheed Martin', 'Dassault Aviation', 'Bombardier Aerospace',
       'Gulfstream Aerospace', 'Cirrus Aircraft'], dtype=object)

In [6]:
! grep ',' dataset/data/images_manufacturer_train.txt

In [7]:
#Recherche le caractère T dans le fichier et n'affiche que trois lignes (head -3)
! grep 'T' dataset/data/images_manufacturer_train.txt | head -3

0724121 ATR
0619697 ATR
2243949 ATR


In [8]:
#wc : Compte le nombre d'élements (-l : ligne, -c : caractère, -w : word)
! grep 'T' dataset/data/images_manufacturer_train.txt | wc 

    132     264    1848


In [9]:
!cut -f 1 -d' ' dataset/data/images_manufacturer_train.txt | head

1025794
1340192
0056978
0698580
0450014
1042824
0894380
1427680
0817494
0716386


In [10]:
manufacturer_df = pd.read_csv(DATA_DIR / 'images_manufacturer_train.txt',sep='\t',
            names=['all'],
            dtype={'all':str}) #ids are not int but string 


#La fonction split() découpe sur une chaîne de caractères 
manufacturer_df['image_id'] = manufacturer_df['all'].apply(lambda x:x.split(' ')[0])  

#La fonction '<car>'.join(liste) concatène les éléments de liste en utillisant le séparateur <car>
manufacturer_df['manufacturer']=manufacturer_df['all'].apply(lambda x:' '.join(x.split(' ')[1:])) 
manufacturer_df['manufacturer'].unique()

array(['Boeing', 'Airbus', 'ATR', 'Antonov', 'British Aerospace',
       'Beechcraft', 'Lockheed Corporation', 'Douglas Aircraft Company',
       'Canadair', 'Cessna', 'McDonnell Douglas', 'de Havilland', 'Robin',
       'Dornier', 'Embraer', 'Eurofighter', 'Lockheed Martin',
       'Dassault Aviation', 'Fokker', 'Bombardier Aerospace',
       'Gulfstream Aerospace', 'Ilyushin', 'Fairchild', 'Piper',
       'Cirrus Aircraft', 'Saab', 'Supermarine', 'Panavia', 'Tupolev',
       'Yakovlev'], dtype=object)

Verify data 

In [11]:
manufacturer_df['manufacturer'].value_counts(dropna=False)

Boeing                      733
Airbus                      434
Embraer                     233
McDonnell Douglas           232
de Havilland                167
Canadair                    134
Douglas Aircraft Company    133
Cessna                      133
British Aerospace           133
Fokker                      100
Lockheed Corporation         68
Beechcraft                   67
Saab                         67
Gulfstream Aerospace         67
Dassault Aviation            67
Tupolev                      66
ATR                          66
Panavia                      34
Yakovlev                     34
Lockheed Martin              34
Dornier                      34
Antonov                      34
Bombardier Aerospace         33
Ilyushin                     33
Fairchild                    33
Piper                        33
Cirrus Aircraft              33
Supermarine                  33
Robin                        33
Eurofighter                  33
Name: manufacturer, dtype: int64

Verify missing data

In [12]:
manufacturer_df.isna().sum()

all             0
image_id        0
manufacturer    0
dtype: int64

In [13]:
assert manufacturer_df['image_id'].isna().sum()==0,"Missing value in image_id"
assert manufacturer_df['manufacturer'].isna().sum()==0,"Missing value in manufacturer"

In [15]:
manufacturer_df.head()

,all,image_id,manufacturer
0,1025794 Boeing,1025794,Boeing
1,1340192 Boeing,1340192,Boeing
2,0056978 Boeing,0056978,Boeing
3,0698580 Boeing,0698580,Boeing
4,0450014 Boeing,0450014,Boeing


In [16]:
manufacturer_df['path'] = manufacturer_df['image_id'].apply(lambda x: pathlib.Path('dataset/data/images') / (x + '.jpg'))

In [17]:
manufacturer_df.head()

,all,image_id,manufacturer,path
0,1025794 Boeing,1025794,Boeing,dataset/data/images/1025794.jpg
1,1340192 Boeing,1340192,Boeing,dataset/data/images/1340192.jpg
2,0056978 Boeing,0056978,Boeing,dataset/data/images/0056978.jpg
3,0698580 Boeing,0698580,Boeing,dataset/data/images/0698580.jpg
4,0450014 Boeing,0450014,Boeing,dataset/data/images/0450014.jpg


In [19]:
def build_image_database(path):
  """Build a pandas dataframe with target class and access path to images. 
  
  Parameters 
  ----------
  path (Path) : path pattern to read csv file containing images information.
  
  Returns 
  -------
  pandas dataframe, including target class and path to image. 

  """
  manufacturer_df = pd.read_csv(path,sep='\t',
            names=['all'],
            dtype={'all':str}) #ids are not int but string 


  #La fonction split() découpe sur une chaîne de caractères 
  manufacturer_df['image_id'] = manufacturer_df['all'].apply(lambda x:x.split(' ')[0])  

  #La fonction '<car>'.join(liste) concatène les éléments de liste en utillisant le séparateur <car>
  manufacturer_df['manufacturer']=manufacturer_df['all'].apply(lambda x:' '.join(x.split(' ')[1:])) 

  #La colonne path contient le chemin d'accès à l'image
  manufacturer_df['path'] = manufacturer_df['image_id'].apply(lambda x: pathlib.Path('dataset/data/images') / (x + '.jpg'))

  return manufacturer_df

In [20]:
build_image_database(DATA_DIR / 'images_manufacturer_train.txt')

,all,image_id,manufacturer,path
0,1025794 Boeing,1025794,Boeing,dataset/data/images/1025794.jpg
1,1340192 Boeing,1340192,Boeing,dataset/data/images/1340192.jpg
2,0056978 Boeing,0056978,Boeing,dataset/data/images/0056978.jpg
3,0698580 Boeing,0698580,Boeing,dataset/data/images/0698580.jpg
4,0450014 Boeing,0450014,Boeing,dataset/data/images/0450014.jpg
...,...,...,...,...
3329,1577680 Yakovlev,1577680,Yakovlev,dataset/data/images/1577680.jpg
3330,1826676 Yakovlev,1826676,Yakovlev,dataset/data/images/1826676.jpg
3331,0472681 Yakovlev,0472681,Yakovlev,dataset/data/images/0472681.jpg
3332,1597829 Yakovlev,1597829,Yakovlev,dataset/data/images/1597829.jpg


In [21]:
# mieux que rien 
def prt():
  """ je fais ceci dans ma fonction
  """
  return 

In [30]:
def build_image_database(path,target):
  """Build a pandas dataframe with target class and access path to images. 
  
  Parameters 
  ----------
  path (Path) : path pattern to read csv file containing images information.
  target (str) : name of the target column 
  Returns 
  -------
  pandas dataframe, including target class and path to image. 

  """
  _df = pd.read_csv(path,sep='\t',
            names=['all'],
            dtype={'all':str}
            ) #ids are not int but string 

  #La fonction split() découpe sur une chaîne de caractères 
  _df['image_id'] = _df['all'].apply(lambda x:x.split(' ')[0])  

  #La fonction '<car>'.join(liste) concatène les éléments de liste en utillisant le séparateur <car>
  _df[target]= _df['all'].apply(lambda x:' '.join(x.split(' ')[1:])) 

  #La colonne path contient le chemin d'accès à l'image
  _df['path'] = _df['image_id'].apply(lambda x: pathlib.Path('dataset/data/images') / (x + '.jpg'))

  return _df.drop(columns=['all'])

In [31]:
build_image_database(DATA_DIR / 'images_manufacturer_train.txt','manufacturer').head(2)

,image_id,manufacturer,path
0,1025794,Boeing,dataset/data/images/1025794.jpg
1,1340192,Boeing,dataset/data/images/1340192.jpg


In [32]:
build_image_database(DATA_DIR / 'images_manufacturer_train.txt','family').head(2)

,image_id,family,path
0,1025794,Boeing,dataset/data/images/1025794.jpg
1,1340192,Boeing,dataset/data/images/1340192.jpg


In [33]:
build_image_database(DATA_DIR / 'images_manufacturer_train.txt','variant').head(2)

,image_id,variant,path
0,1025794,Boeing,dataset/data/images/1025794.jpg
1,1340192,Boeing,dataset/data/images/1340192.jpg


In [37]:
def build_image_database(path, target) :
  """Build a pandas dataframe with target class and access path to images.

  Parameters
  ----------
  path (Path) : path pattern to read csv file containing images information. 
  target (str) : name of the target column.

  Returns 
  -------
  A pandas dataframe, including target class and path to image.
  """

  _df =pd.read_csv(path, sep='\t'
                , names=['all']
                , dtype ={'all':str} # id are string not int
                )

  #split donne une chaine de caractères
  _df['image_id'] = _df['all'].apply(lambda x: x.split(' ')[0])

  #'<car>'.join(liste) concatène les éléments de la liste avec le séparateur '<car>'
  _df[target] = _df['all'].apply(lambda x: ' '.join(x.split(' ')[1:]))

  # La colonne path contient le chemin d'accès à l'image
  _df['path'] = _df['image_id'].apply(lambda x: pathlib.Path('dataset/data/images') / (x +'.jpg'))

  return _df.drop(columns=['all'])

In [38]:
build_image_database(DATA_DIR / 'images_manufacturer_train.txt','variant').head(2)

,image_id,variant,path
0,1025794,Boeing,dataset/data/images/1025794.jpg
1,1340192,Boeing,dataset/data/images/1340192.jpg


baselib : module qui viennent systématiquement avec Python, 

import os 
import pathlib 
import datetime 